**Load the required Packages**


In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import os

**Fill in the command for reading the CSV dataset "HandsOnRFM.csv" using pandas**

In [2]:
df =









**Following cell helps in doing the Data PreProcessing. Run the Cell to get it done for your Dataset**

In [3]:
#find the coutry in which each of the customer has made purchase
customer_country=df[['Country','CustomerID']].drop_duplicates()
customer_country
#remove null values
df = df[pd.notnull(df['CustomerID'])]
#removes negative values
df.Quantity.min()
df = df[(df['Quantity']>0)]
df.Quantity.min()
#computes totalprice for monetary calculation
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
NOW = dt.datetime(2011,12,10)
#changing date format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])



**Following Cell commands helps in calculation of Recency, Frequency and Monetary Value.**
**Calculation of Recency, Frequency, Monetary Value is done by the following procedure**
- Recency value found by by subtracting that customer's maximum Invoice Date from NOW
- Frequency value found by counting the Invoice Number of that customer.
- Monetary value found by adding up all the totalprice of purchases of that Customer.

**Run the cell to see the output**


In [24]:
#calculate the recency, frequency and monetary_value

rfmTable=df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})

##Renaming columns

rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)
rfmTable





**Following cell helps in casting the values to integer. Run the cell to see the output**

In [17]:

rfmTable['recency'] = rfmTable['recency'].astype(int)
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()
segmented_rfm = rfmTable



**Below cell contains the function to calculates the R score. Run the cell to define the function such that it can be applied on the dataset.**

In [11]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

**Write a function FMScore which calculates the Frequency and Monetary ranking**
- Remember that difference between Rscore and FMScore is that in the former the lower quartile will be assigned 1 rank whereas in the latter greatest quartile will be assigned 1 rank.

In [19]:
#FMScore calculation

def FMScore(x,p,d):   

    
    
    
    
    
    
    
    

- **Following cell contains the command to apply the functions R score and FMScore on the dataset.**
- **Run the cell to calculate R Score F Score and M Score for your Dataset**

In [25]:
#R score calculation

segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
# F Score calculation
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
#M score calculation
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))


R= np.asarray(segmented_rfm[['r_quartile']])
R_mean=np.mean(R);
F= np.asarray(segmented_rfm[['f_quartile']])
F_mean=np.mean(F);
M= np.asarray(segmented_rfm[['m_quartile']])
M_mean=np.mean(M);
final=R_mean+F_mean+M_mean
with open('output1.txt', 'w') as file:
    file.write(str(final))